In [846]:
import numpy as np
import pandas as pd
import scipy as sc
import sklearn as skl
import nltk
import os
import io
import re
import csv
import operator
import warnings
from nltk.corpus import stopwords
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from pandas import *
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from elasticsearch import helpers
from elasticsearch.helpers import bulk
warnings.filterwarnings('ignore')

In [595]:
# SET STOP WORDS

stop = set(stopwords.words('english'))
stop.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}'])
#companiesTraining = pd.read_csv('D:\\Homework3\\all\companies.csv', delimiter=',', encoding='latin-1', header=None, names=["Companies Training"])

In [120]:
# OPEN NEWS ARTICLES INTO PYTHON MEMORY

######################################################################################################################
################################### INSERT LOCATION OF 2013 AND 2014 FOLDERS HERE #####################################
########################################################################################################################
texts = []
location = 'D:\\HW4\\2013'
for filename in os.listdir(location):
    fullfilename = location + '\\' + filename
    with open(fullfilename, 'r', encoding='latin-1') as myfile:
        text = myfile.read()
        text = nltk.sent_tokenize(text)
        for i in np.arange(len(text)):
            texts.append(text[i])
        myfile.close()

location = 'D:\\HW4\\2014'
for filename in os.listdir(location):
    fullfilename = location + '\\' + filename
    with open(fullfilename, 'r', encoding='latin-1') as myfile:
        text = myfile.read()
        text = nltk.sent_tokenize(text)
        for i in np.arange(len(text)):
            texts.append(text[i])
        myfile.close()

In [156]:
# TOKENIZE ARTICLES INTO SENTENCES AND INDEX INTO ELASTICSEARCH

client = Elasticsearch()
actions = []

for j in range(0,len(texts)):
    currSentence = texts[j]
    action = {
        "_index": "articles-index",
        "_type": "articles",
        "_id": j,
        "_source": {
            "any": "data" + str(j),
            "timestamp": datetime.now(),
            "body": currSentence
        }
    }
    actions.append(action)
    
if len(actions) > 0:
    helpers.bulk(client, actions)

In [395]:
# EXTRACT_ENTITIES ACCEPTS A SENTENCE AND OUTPUTS THE NAMED-ENTITIES IN A LIST

def extract_entities(sent):
    if type(sent) == str:
        namedEntities = ne_chunk(pos_tag(word_tokenize(sent)))
    else:
        namedEntities = ne_chunk(pos_tag(sent))
    continuous = []
    current = []
    
    for i in namedEntities:
        if type(i) == Tree:
            current.append(" ".join([token for token, pos in i.leaves()]))
        elif current:
            named_entity = " ".join(current)
            if named_entity not in continuous:
                continuous.append(named_entity)
                current = []
        else:
            continue
            
    return continuous

In [817]:
# QUESTION CLASSIFICATION
# Function that accepts a sentence string as input and outputs question type

# 1: Which companies went bankrupt in month X of year Y?
# 2: What affects GDP?
# 3: What percentage of drop or increase is associated with this property?
# 4: Who is the CEO of company X?

def classifyQuestion(candidate) :
    '''
    Returns 0, 1, 2, or 3 corresponding to question type 1, 2, 3, or 4
    '''
    question1 = "which companies went bankrupt in month x of year y?"
    question2 = "what affects gdp?"
    question3 = "what percentage % of drop or increase change in gdp is associated with results from this property consumption consumer spending government investment imports exports foreign trade?"
    question4 = "who is the ceo of company x?"

    questions = []
    questions.append(question1)
    questions.append(question2)
    questions.append(question3)
    questions.append(question4)
    
    candidate = candidate.lower()
    candidateTokenized = nltk.word_tokenize(candidate)
    questionsTokenized = []
    cosineSimilarity = np.zeros(4)
    candidateNorm = np.sqrt(len(candidateTokenized))

    for i in np.arange(len(questions)):
        questionsTokenized.append(nltk.word_tokenize(questions[i]))
        questionNorm = np.sqrt(len(questionsTokenized[i]))
        match = 0
        for j in np.arange(len(candidateTokenized)):
            if (candidateTokenized[j] in questionsTokenized[i]):
                match = match + 1
        similarity = match / (candidateNorm * questionNorm)
        cosineSimilarity[i] = similarity

    index_max = np.argmax(cosineSimilarity)
    return(index_max)

In [850]:
# ANSWER A GDP PERCENTAGE QUESTION
# Accepts question as input and outputs the percentage effect on GDP

def gdpPercent(question):
    questionToken = nltk.word_tokenize(question)
    if "?" in questionToken:
        questionToken.remove("?")
    questionPOS = nltk.pos_tag(questionToken)

    target = []
    for i in range(len(questionToken)-1, -1, -1):
        if questionPOS[i][1] != 'TO' and questionPOS[i][1] != 'IN':
            target.insert(0, questionToken[i])
        else:
            break

    query = ['GDP', 'affect', 'effect', 'effects','affects', 'decrease', 'increase', 'drag', '%', 'percent', 'growth']
    for i in range(0,len(target)):
        query.append(target[i])
    gdpSearch = client.search(index="articles-index", q=query, size=1000)
    gdpHits = gdpSearch['hits']['hits']

    candSentences = []
    candTokenized = []
    candScore = []
    check = query
    check.remove('GDP')
    for i in np.arange(len(gdpHits)):
        sentence = gdpHits[i]['_source']['body']
        sentenceToken = nltk.word_tokenize(sentence)
        score = 0
        if 'GDP' in sentenceToken:
            score = score + 2
        for i in np.arange(len(target)):
            if target[i] in sentenceToken:
                score = score + 2
        for i in np.arange(len(check)):
            if check[i] in sentenceToken:
                score = score + 1
        candSentences.append(sentence)
        candTokenized.append(sentenceToken)
        candScore.append(score)

    candDF = pd.DataFrame(data=[candSentences,candTokenized,candScore], index=['Sent','Token','Score'])
    candDF = candDF.transpose()
    candDF = candDF.sort(['Score'], ascending=False)
    candDF.index = range(0,len(candDF['Sent']))

    percentages = ['%','percent','percentage']
    topsent = []
    topscores = []
    topanswers = []
    for i in range(0,3):
        topsent.append(candDF['Token'][i])
        score = np.zeros(len(topsent[i]))
        for j in range(0,len(score)):
            gdpLeft = 100
            gdpRight = 100
            targLeft = 100
            targRight = 100
            ispercent = 0
            if topsent[i][j] in percentages:
                ispercent = 1
            count = 0
            for k in range(j, -1, -1):
                if topsent[i][k] == "GDP":
                    gdpLeft = count
                    break
                else:
                    count = count + 1
            count = 0
            for k in range(j, -1, -1):
                if topsent[i][k] == target[0]:
                    targLeft = count
                    break
                else:
                    count = count + 1           
            count = 0
            for k in range(j, len(topsent[i]), 1):
                if topsent[i][k] == "GDP":
                    gdpRight = count
                    break
                else:
                    count = count + 1
            count = 0
            for k in range(j, len(topsent[i]), 1):
                if topsent[i][k] == target[0]:
                    targRight = count
                    break
                else:
                    count = count + 1
            gdpDist = min([gdpLeft, gdpRight])
            targDist = min([targLeft, targRight])
            score[j] = ispercent * 500 - gdpDist - targDist
        topscores.append(score)
        index, value = max(enumerate(topscores[i]), key=operator.itemgetter(1))
        answer = [topsent[i][index-1], topsent[i][index]]
        topanswers.append(answer)

    for i in range(0,3):
        if topanswers[i][1] in percentages:
            topanswers[i][1] = "percent"
            output = topanswers[0][0] + " "
            output = output + topanswers[0][1]
            break
    return output

In [636]:
# ANSWER A BANKRUPTCY QUESTION
# Accepts question as input and returns the name of the company that went bankrupt

def companyBankrupt(question):
    questionToken = nltk.word_tokenize(question)
    questionToken = [word for word in questionToken if not word in stop]
    questionToken = nltk.pos_tag(questionToken)
    query = ['bankrupt', 'bankruptcy', 'chapter 7', 'chapter 11', 'chapter 13', 'filed', 'declared']
    date = []
    for i in range(0,len(questionToken)):
        pos = questionToken[i][1]
        if (pos == 'NNP') or (pos == 'CD'):
            query.append(questionToken[i][0])
            date.append(questionToken[i][0])

    query.append
    bankruptSearch = client.search(index="articles-index", q=query, size=5000)
    candSentences = []
    candTokenized = []
    searchHits = bankruptSearch['hits']['hits']
    for i in np.arange(len(searchHits)):
        currSentence = searchHits[i]['_source']['body']
        currTokenized = nltk.word_tokenize(currSentence)
        currTokenized = [word for word in currTokenized if not word in stop]

        includeDate = 1
        includeBank = 0
        Bank = ['bankruptcy', 'bankrupt', 'Chapter']
        for i in range(0,len(date)):
            if date[i] not in currTokenized:
                includeDate = 0
        for i in range(0, len(Bank)):
            if Bank[i] in currTokenized:
                includeBank = 1
        if (includeDate == 1) and (includeBank == 1):
            candSentences.append(currSentence)
            candTokenized.append(currTokenized)

    candidates = []
    for i in np.arange(len(candSentences)):
        candEntities = extract_entities(candSentences[i])
        for j in np.arange(len(candEntities)):
            candidates.append(candEntities[j])
    
    if candidates != []:
        return max(set(candidates), key=candidates.count)
    else:
        print("Sorry, I could not find what you were looking for.")
        return -1       

In [430]:
# ANSWER A CEO NAME QUESTION
# Accepts question as input and returns the name of that company's CEO

def findCEO(question):
    companyQuery = extract_entities(question)
    if companyQuery == []:
        print("Sorry, I could not find what you were looking for.")
        return -1
    CEOsearch = client.search(index="articles-index", q=companyQuery, size=100)
    candSentences = []
    for i in np.arange(len(CEOsearch['hits']['hits'])):
        candSentences.append(CEOsearch['hits']['hits'][i]['_source']['body'])
        candSentences[i] = nltk.word_tokenize(candSentences[i])
    tokenQuestion = nltk.word_tokenize(companyQuery[0])
    if "of" in tokenQuestion:
        tokenQuestion.remove('of')
    if len(tokenQuestion) == 1:
        companyName = tokenQuestion[0]
    else:
        companyName = tokenQuestion[1]
    if len(companyName) > 2:
        for i in range(2, len(tokenQuestion)):
            companyName = companyName + " "
            companyName = companyName + tokenQuestion[i]
        
    reducedSentences = []
    for i in np.arange(len(candSentences)):
        keep = 1
        for j in np.arange(len(tokenQuestion)):
            if tokenQuestion[j] not in candSentences[i]:
                keep = 0
        if keep == 1:
            reducedSentences.append(candSentences[i])
            
    candNames = []
    for i in np.arange(len(reducedSentences)):
        candEntities = extract_entities(reducedSentences[i])
        for j in np.arange(len(candEntities)):
            if (candEntities[j] != companyName) and (len(nltk.word_tokenize(candEntities[j]))==2):
                candNames.append(candEntities[j])
    
    if candNames != []:
        return max(set(candNames), key=candNames.count)
    else:
        print("Sorry, I could not find what you were looking for.")
        return -1

In [810]:
# ANSWER QUESTION
# Function accepts a sentence as input, runs the question classifier, and
# selects the appropriate answering algorith to run

def answerMyQuestion(candidate) :
    questionType = classifyQuestion(candidate)
    if questionType == 0 :
        return(companyBankrupt(candidate))
    elif questionType == 1 :
        return("Consumption, consumer spending, government spending, investment, imports, exports, foreign trade")
    elif questionType == 2 :
        return(gdpPercent(candidate))
    elif questionType == 3 :
        return(findCEO(candidate))

In [821]:
answerMyQuestion("Which company went bankrupt in September 2008?")

'Lehman Brothers'

In [830]:
answerMyQuestion("Which companies went bankrupt in July 2013?")

'Detroit'

In [832]:
answerMyQuestion("Which companies filed for bankruptcy in November of 2011?")

'US Airways'

In [834]:
answerMyQuestion("What factors most affect GDP?")

'Consumption, consumer spending, government spending, investment, imports, exports, foreign trade'

In [851]:
answerMyQuestion("What percentage drop or increase is associated with exports?")

'1.5 percent'

In [852]:
answerMyQuestion("What percentage is associated with foreign trade?")

'0.7 percent'

In [853]:
answerMyQuestion("What change in GDP results from consumer spending?")

'2 percent'

In [843]:
answerMyQuestion("Who is the CEO of Tesla?")

'Elon Musk'

In [844]:
answerMyQuestion("Who is the CEO of Facebook?")

'Mark Zuckerberg'

In [855]:
answerMyQuestion("Who is the CEO at ZS Associates?")

'Rich Handler'